# Verificação Indutiva

Na aula passada vimos como modelar sistemas dinâmicos com *First-order Transition Systems* (FOTSs) e como verificar propriedades de segurança e animação sobre estes sistemas com *Bounded Model Checking* (BMC). O procedimento de BMC verifica se uma propriedade é válida num FOTS considerando apenas um número limitado de estados de execução. Se quisermos verificar uma propriedade para qualquer execução não limitada do FOTS temos que usar um procedimento alternativo. Nesta aula vamos usar indução para fazer verificação em traços não limitadados.

## Verificação indutiva de invariantes

No caso da verificação de propriedades de segurança $G\ \phi$, para verificar o invariante $\phi$ por indução temos que verificar as seguintes condições:
- $\phi$ é válido nos estados iniciais, ou seja, $\mathit{init}(s) \rightarrow \phi(s)$
- Para qualquer estado, assumindo que $\phi$ é verdade, se executarmos uma transição, $\phi$ continua a ser verdade no próximo estado, ou seja, $\phi(s) \wedge \mathit{trans}(s,s') \rightarrow \phi(s')$.

### Exercício 1

Implemente a função `induction_always` para verificação de invariantes por indução. 
A função recebe como argumento uma função que gera uma cópia das variáveis do estado, um predicado que testa se um estado é inicial, um predicado que testa se um par de estados é uma transição válida, e o invariante.

Note que terá que testar a validade das duas condições acima recorrendo à satisfiabilidade, ou seja, usando o solver para encontrar contra-exemplos, devendo o procedimento reportar qual das propriedades falha. Por exemplo, no caso da primeira deve procurar uma valoração que satisfaça $\mathit{init}(s) \wedge \neg \phi(s)$.

In [ ]:
from pysmt.shortcuts import *
from pysmt.typing import INT


def induction_always(declare,init,trans,inv):
    with Solver(name="z3") as s:
    
    # completar
    

Usando este procedimento podemos tentar verificar a propriedade $G (x \ge 0)$ sobre o programa

```Python
{ x >= 3 }
0: while (x>0):
1:    x = x-1
2: stop
```

In [ ]:
def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i),INT)
    state['x'] = Symbol('x'+str(i),INT)
    return state

def init(state):
    return And(Equals(state['pc'],Int(0)), state['x']>= Int(3))

def trans(curr,prox):
    t01 = And(Equals(curr['pc'],Int(0)),curr['x'] > Int(0), Equals(prox['pc'],Int(1)), Equals(prox['x'],curr['x']))
    t02 = And(Equals(curr['pc'],Int(0)),curr['x'] <= Int(0), Equals(prox['pc'],Int(2)), Equals(prox['x'],curr['x']))
    t10 = And(Equals(curr['pc'],Int(1)),Equals(prox['pc'],Int(0)),Equals(prox['x'],curr['x']-Int(1)))
    t22 = And(Equals(curr['pc'],Int(2)),Equals(prox['pc'],Int(2)),Equals(prox['x'],curr['x']))
    return Or(t02,t01,t10,t22)

def nonnegative(state):
    return (state['x'] >= Int(0))

induction_always(declare,init,trans,nonnegative)

Como se pode verificar, a propriedade anterior, embora seja verdade, não pode ser verificada por indução, pois o passo indutivo falha quando $\mathit{pc} = 1$ e $x = 0$. De facto, este contra-exemplo para o passo de indução não é realista, dado que este estado não é acessível a partir do estado inicial. Para verificarmos esta propriedade por indução temos que encontrar um invariante $\phi$ que seja mais forte que o desejado, ou seja, $\phi(x,\mathit{pc}) \rightarrow x \ge 0$, e que seja verificável por indução.

### Exercício 2

Encontre uma propriedade mais forte que $x \ge 0$ e que seja verificável por indução. Sugestão: pode ser útil usar a funcção `Ite` do pySMT, que corresponde a um if-then-else lógico.

In [ ]:
# completar


### Exercício 3

Utilize o SMT solver para verificar que essa propriedade é realmente mais forte que $x \ge 0$.

In [ ]:
# completar


Nem sempre é fácil encontrar um invariante indutivo mais forte que o desejado. Um procedimento de verificação alternativo que pode ser utilizado nesses casos é a designada $k$-indução. A ideia é generalizar a indução simples assumindo no passo indutivo que o invariante é válido nos $k$ estados anteriores. Claro que neste caso também será necessário verificar que a propriedade é válida nos $k$ primeiros estados. Com esta generalização é possível verificar que $x \ge 0$ é um invariante recorrendo a $k$-indução com $k=2$.

### Exercício 4

Implemente o método `kinduction_always` para verificação de invariantes por $k$-indução.

In [ ]:
def kinduction_always(declare,init,trans,inv,k):

    # completar


## Verificação de propriedades de animação por indução

Como vimos, podemos verificar propriedades de animação do tipo $F\ \phi$ usando BMC. Mais uma vez, se quisermos verificar estas propriedades para qualquer execução ilimitada temos que usar um procedimento alternativo. Uma possibilidade consiste em reduzir a verificação dessas propriedades à verificação de uma propriedade de segurança, mais concretamente um invariante, que possa ser verificado por indução.

Por exemplo, uma técnica que pode ser usada para verificar uma propriedade de animação do tipo $F\ (G\ \phi)$, passa por descobrir um *variante* $V$ que satisfaz as seguintes condições:
- O variante nunca é negativo, ou seja, $G\ (V(s) \ge 0)$
- O variante descresce sempre (estritamente) ou atinge o valor 0, ou seja, $G\ (\forall s' . \mathit{trans}(s,s') \rightarrow (V(s') < V(s) \vee V(s') = 0))$
- Quando o variante é 0 verifica-se necessariamente $\phi$, ou seja, $G\ (V(s)=0 \rightarrow \phi(s))$

A terminação de um programa é um dos exemplos de propriedade que encaixa neste padrão. Por exemplo, para o programa acima a terminação pode ser expressa por $F\ (G\ \mathit{pc}=2)$ (note que devido ao lacete no estado final, o programa permanece indefinidamente no estado final).

Note que a segunda condição recorre a uma quantificação universal sobre os possíveis estados alcançáveis por uma transição a partir de cada estado.

### Exercício 5

Descubra o variante que permite provar por indução que o programa acima termina. Encontre também os menores valores de $k$ que permitem provar as condições acima por $k$-indução.

In [ ]:
# completar

### Exercício 6

Encontrar um variante que decresça em todas as transições nem sempre é fácil. Podemos relaxar a segunda condição acima e exigir que o variante apenas tenha que decrescer estritamente a cada $l$ transições. Neste caso dizemos que temos um *lookahead* de $l$. Altere a definição da função decrescente por forma a considerar um lookahead de 2 e encontre um variante mais simples que o anterior que lhe permita verificar a terminação do programa.

In [ ]:
# completar